# AgentOps Basic Monitoring 
This is an example of how to use the AgentOps library for basic Agent monitoring with OpenAI's GPT

In [1]:
from openai import OpenAI
import agentops
import os
from dotenv import load_dotenv

Next, we'll grab our two API keys. You can use dotenv like below or however else you like to load environment variables

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

The AgentOps library is designed to be a plug-and-play replacement for the OpenAI Client, maximizing use with minimal install effort.

In [3]:
openai = OpenAI(api_key=OPENAI_API_KEY)
agentops.init(AGENTOPS_API_KEY, tags=['openai-gpt-notebook'])

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=806d7e02-abae-4e17-9c0e-b84c30157615


Now just use OpenAI as you would normally!

## Single Session with ChatCompletion

In [4]:
message = ({"role": "user", "content": "Hello"},)

generator = openai.chat.completions.create(
    model="gpt-3.5-turbo", messages=message, temperature=0.5
)

Make sure to end your session with a `Result` (Success|Fail|Indeterminate) for better tracking

In [5]:
agentops.end_session("Success")

🖇 AgentOps: This run's cost $0.000017
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=806d7e02-abae-4e17-9c0e-b84c30157615


[{'event_type': 'llms', 'params': {'model': 'gpt-3.5-turbo', 'messages': ({'role': 'user', 'content': 'Hello'},), 'temperature': 0.5}, 'returns': {'id': 'chatcmpl-9etn3UDr2IpCwEQlCAoOa7LhDWXtq', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'Hello! How can I assist you today?', 'role': 'assistant', 'function_call': None, 'tool_calls': None}}], 'created': 1719534605, 'model': 'gpt-3.5-turbo-0125', 'object': 'chat.completion', 'system_fingerprint': None, 'usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}}, 'init_timestamp': '2024-06-28T00:30:04.218Z', 'end_timestamp': '2024-06-28T00:30:05.421Z', 'agent_id': None, 'id': UUID('b0a1abb7-9ebd-407e-ba24-f612b70eebcf'), 'thread_id': None, 'prompt': ({'role': 'user', 'content': 'Hello'},), 'prompt_tokens': 8, 'completion': {'content': 'Hello! How can I assist you today?', 'role': 'assistant', 'function_call': None, 'tool_calls': None}, 'completion_tokens': 9, 'model': 'gpt-3.5-

Now if you check the AgentOps dashboard, you should see information related to this run!

# Events
Additionally, you can track custom events via AgentOps.
Let's start a new session and record some events 

In [6]:
# Create new session
agentops.start_session(tags=["openai-gpt-notebook-events"])

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=128cc6ae-3bd3-405f-865d-6fbc0b34f4b9


The easiest way to record actions is through the use of AgentOp's decorators

In [7]:
from agentops import record_function


@record_function("add numbers")
def add(x, y):
    return x + y


add(2, 4)

6

We can also manually craft an event exactly the way we want

In [8]:
from agentops import ActionEvent

message = ({"role": "user", "content": "Hello"},)
response = openai.chat.completions.create(
    model="gpt-3.5-turbo", messages=message, temperature=0.5
)

if "hello" in str(response.choices[0].message.content).lower():
    agentops.record(
        ActionEvent(
            action_type="Agent says hello",
            params=str(message),
            returns=str(response.choices[0].message.content),
        )
    )

In [9]:
agentops.end_session("Success")

🖇 AgentOps: This run's cost $0.000017
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=128cc6ae-3bd3-405f-865d-6fbc0b34f4b9


[{'event_type': 'actions', 'params': {'x': 2, 'y': 4}, 'returns': 6, 'init_timestamp': '2024-06-28T00:30:06.877Z', 'end_timestamp': '2024-06-28T00:30:06.883Z', 'agent_id': None, 'id': UUID('80bbb402-84fb-4ebf-a1b7-181109f356ea'), 'action_type': 'add numbers', 'logs': None, 'screenshot': None}, {'event_type': 'llms', 'params': {'model': 'gpt-3.5-turbo', 'messages': ({'role': 'user', 'content': 'Hello'},), 'temperature': 0.5}, 'returns': {'id': 'chatcmpl-9etn5beqXFWcKcYjN7cz8KfjV9nkr', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'Hello! How can I assist you today?', 'role': 'assistant', 'function_call': None, 'tool_calls': None}}], 'created': 1719534607, 'model': 'gpt-3.5-turbo-0125', 'object': 'chat.completion', 'system_fingerprint': None, 'usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}}, 'init_timestamp': '2024-06-28T00:30:06.887Z', 'end_timestamp': '2024-06-28T00:30:07.500Z', 'agent_id': None, 'id': UUID('a9cd45d